# LAUNCH ME

Following this file, you will be able to see our experiments and reproduce the result we shown in our project.

## 0. Import

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import kagglehub


# Our implemented parallelized class from the paper Dictionary Learning for Sparse Coding
from src import DictionaryAlgoParallel 

# This function will import the dataset we need
from src import get_boat_data, get_instruments_data, get_berekley_data_loaders 



## I. Boat dataset

## I.1 Hyperparameters

Lets first import the dataset from kaggle and parse it.

In [2]:
from src import plot_image_grid

def rescale_patch(patch, new_size=(16,16)):
    patch_uint8 = (patch * 255).clip(0, 255).astype(np.uint8) if patch.dtype != np.uint8 else patch
    pil_img = Image.fromarray(patch_uint8)
    downscaled_image = pil_img.resize(new_size, Image.Resampling.LANCZOS)
    downscaled_np = np.array(downscaled_image)
    return downscaled_np


path_boat = kagglehub.dataset_download("rhammell/ships-in-satellite-imagery")

print("Path to dataset files:", path_boat)
x,y_true, images, jsondata = get_boat_data(path_boat)

x = np.array(list(map(rescale_patch, x)))

Path to dataset files: /home/antoine/.cache/kagglehub/datasets/rhammell/ships-in-satellite-imagery/versions/9


Now that we have our dataset, we can check what it contains.

In [ ]:
def preprocessing(datas):
    """
    Flatten our data.
    :param datas: (n_samples, n_rows, n_cols, n_channels)
    :return: 
    """
    n_samples, n_rows, n_cols, n_channels = datas.shape
    datas = datas.reshape(n_samples, -1) / 255
    preprocessed = datas
    infos = {"shape" : (n_rows, n_cols)}
    return  preprocessed, infos

def postprocessing(datas, infos, ):
    """
    Unflatten our data to reconstruct our patch.
    """
    n_rows, n_cols = infos["shape"]
    results = datas
    results = results.reshape(len(datas), n_rows, n_cols, 3, order="C").clip(0,1)
    return results

x_boat = x[y_true == 1.]
x_background = x[y_true == 0.]
x_background_preprocessing, infos_background_preprocessing = preprocessing(x_background)
x_boat_preprocessing, infos_boat_preprocessing = preprocessing(x_boat)

print(" training data shape", x_background_preprocessing.shape)
print(" boat (not used in training) data shape", x_boat_preprocessing.shape)



In [ ]:
plot_image_grid(x_boat, 2, figsize = (2,2))

In [ ]:
plot_image_grid(x_background, 2, figsize = (2,2))

We can now train our Dictionary using our background preprocessed data

In [ ]:
def get_trained_dico():
    m = patch_size[0] * patch_size[1]*3
    nb_atoms = 30
    dict_learner = DictionaryAlgoParallel(m=m, k=nb_atoms, lbd=1e-2, dic_update_steps=2, use_cuda=False, dico_update="quick_update", n_jobs=-1, verbose=True) # Initialize our python class containing our training logic
    
    from tqdm import tqdm
    from itertools import cycle
    class FiniteGenerator:
        def __init__(self, iterable, nb_elt,  batch_size = 8):
            self.iterable = iterable
            self.nb_elt = nb_elt
            self.batch_size = batch_size
        def __len__(self):
            return self.nb_elt
        def __iter__(self,):
            infinite_loader = cycle(self.iterable)          
            for k in tqdm(range(self.nb_elt // self.batch_size)):
                data = np.array([next(infinite_loader) for _ in range(self.batch_size)])
                yield torch.from_numpy(data).float()
                
    x_training = FiniteGenerator(x_background_preprocessing, 4*len(x_boat_preprocessing), batch_size=32)
    dict_learner.D = torch.tensor(torch.from_numpy(x_background_preprocessing[:nb_atoms].T), dtype=torch.float)
    dict_learner.D.to(dict_learner.device)
    dict_learner.fit(x_training)
    return dict_learner

## II. Instrument dataset

In [ ]:
path_instrument = kagglehub.dataset_download("soumendraprasad/musical-instruments-sound-dataset")

print("Path to dataset files:", path_instrument)

Ys, sampling_rates, files = get_instruments_data(path_instrument)

def to_mono(s):
    if len(s.shape) == 2:  # Stereo signal
        return np.mean(s, axis=1)  # Average the two channels to get mono
    return s

signals = list(map(to_mono, Ys))

## III Berkeley image dataset

### III.1 Hyperparameters


In [ ]:
patch_size = (12,12)
batch_size = ...

### III.2 Import dataset


In [ ]:

path_berkeley = kagglehub.dataset_download("balraj98/berkeley-segmentation-dataset-500-bsds500")

print("Path to dataset files:", path_berkeley)

get_berekley_data_loaders("berkeley",  patch_size=patch_size, patches_per_image=5, batch_size=batch_size, norm_type=NormalizationType.ZSCORE)